In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3871f3d5a21230dc617e443e2b48f7ce52d3ff5e752438ec9dc2c0adebc1f506
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import os
from google.colab import drive
from pyspark.sql import functions as F
#from pyspark.sql.functions import explode, col, concat, lit,size, desc, asc, asc_nulls_last, substring,udf,year

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
df=spark.read.csv('gdrive/My Drive/Colab Notebooks/brewery_data_complete_extended.csv',inferSchema=True,header=True)

In [6]:
df.show(n=5, truncate=False, vertical=True)

-RECORD 0-------------------------------------------
 Batch_ID                     | 7870796             
 Brew_Date                    | 2020-01-01 00:00:19 
 Beer_Style                   | Wheat Beer          
 SKU                          | Kegs                
 Location                     | Whitefield          
 Fermentation_Time            | 16                  
 Temperature                  | 24.204250857069873  
 pH_Level                     | 5.2898454476095615  
 Gravity                      | 1.0395041267301979  
 Alcohol_Content              | 5.370842159553436   
 Bitterness                   | 20                  
 Color                        | 5                   
 Ingredient_Ratio             | 1:0.32:0.16         
 Volume_Produced              | 4666                
 Total_Sales                  | 2664.7593448382822  
 Quality_Score                | 8.57701633109399    
 Brewhouse_Efficiency         | 89.19588216376087   
 Loss_During_Brewing          | 4.104987659187

In [7]:
df.printSchema()

root
 |-- Batch_ID: integer (nullable = true)
 |-- Brew_Date: timestamp (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: integer (nullable = true)
 |-- Color: integer (nullable = true)
 |-- Ingredient_Ratio: string (nullable = true)
 |-- Volume_Produced: integer (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)

